# Introduction to Vector

Vector is a Python library for 2D, 3D, and [Lorentz vectors](https://en.wikipedia.org/wiki/Special_relativity#Physics_in_spacetime), especially _arrays of vectors_, to solve common physics problems in a NumPy-like way.

Main features of Vector:

   * Pure Python with NumPy as its only dependency. This makes it easier to install.
   * Vectors may be represented in a variety of coordinate systems: Cartesian, cylindrical, spherical, and any combination of these with time or proper time for Lorentz vectors. In all, there are 12 coordinate systems: {$x$-$y$ vs $\rho$-$\phi$ in the azimuthal plane} × {$z$ vs $\theta$ vs $\eta$ longitudinally} × {$t$ vs $\tau$ temporally}.
   * Uses names and conventions set by [ROOT](https://root.cern/)'s [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html) and [Math::LorentzVector](https://root.cern.ch/doc/master/classROOT_1_1Math_1_1LorentzVector.html), as well as [scikit-hep/math](https://github.com/scikit-hep/scikit-hep/tree/master/skhep/math), [uproot-methods TLorentzVector](https://github.com/scikit-hep/uproot3-methods/blob/master/uproot3_methods/classes/TLorentzVector.py), [henryiii/hepvector](https://github.com/henryiii/hepvector), and [coffea.nanoevents.methods.vector](https://coffeateam.github.io/coffea/modules/coffea.nanoevents.methods.vector.html).
   * Implemented on a variety of backends:
      - pure Python objects
      - NumPy arrays of vectors (as a [structured array](https://numpy.org/doc/stable/user/basics.rec.html) subclass)
      - [Awkward Arrays](https://awkward-array.org/) of vectors
      - potential for more: CuPy, TensorFlow, Torch, JAX...
   * Implemented in [Numba](https://numba.pydata.org/) for JIT-compiled calculations on vectors.
   * Distinction between geometrical vectors, which have a minimum of attribute and method names, and vectors representing momentum, which have synonyms like `pt` = `rho`, `energy` = `t`, `mass` = `tau`.

This notebook requires Vector, NumPy, Awkward Array, and Numba to run.

In [1]:
import vector
import numpy as np
import awkward as ak
import numba as nb

## Constructing a vector or an array of vectors

The easiest way to create one or many vectors is with a helper function:

   * `vector.obj` to make a pure Python vector object,
   * `vector.array` to make a NumPy array of vectors (lowercase, like `np.array`),
   * `vector.Array` to make an Awkward Array of vectors (uppercase, like `ak.Array`).

### Pure Python vectors

In [2]:
vector.obj(x=3, y=4)   # Cartesian 2D vector

vector.obj(x=3, y=4)

In [3]:
vector.obj(rho=5, phi=0.9273)   # same in polar coordinates

vector.obj(rho=5, phi=0.9273)

In [4]:
vector.obj(x=3, y=4).isclose(vector.obj(rho=5, phi=0.9273))   # use "isclose" unless they are exactly equal

True

In [5]:
vector.obj(x=3, y=4, z=-2)   # Cartesian 3D vector

vector.obj(x=3, y=4, z=-2)

In [6]:
vector.obj(x=3, y=4, z=-2, t=10)   # Cartesian 4D vector

vector.obj(x=3, y=4, z=-2, t=10)

In [7]:
vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10)   # in rho-phi-eta-t cylindrical coordinates

vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10)

In [8]:
vector.obj(pt=5, phi=0.9273, eta=-0.39, E=10)   # use momentum-synonyms to get a momentum vector

vector.obj(pt=5, phi=0.9273, eta=-0.39, E=10)

In [9]:
vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10) == vector.obj(pt=5, phi=0.9273, eta=-0.390035, E=10)

False

In [10]:
vector.obj(rho=5, phi=0.9273, eta=-0.39, t=10).tau   # geometrical vectors have to use geometrical names ("tau", not "mass")

8.426194916448265

In [11]:
vector.obj(pt=5, phi=0.9273, eta=-0.39, E=10).mass   # momentum vectors can use momentum names (as well as geometrical ones)

8.426194916448265

In [12]:
vector.obj(pt=5, phi=0.9273, theta=1.9513, mass=8.4262)   # any combination of azimuthal, longitudinal, and temporal coordinates is allowed

vector.obj(pt=5, phi=0.9273, theta=1.9513, mass=8.4262)

In [13]:
vector.obj(x=3, y=4, z=-2, t=10).isclose(vector.obj(pt=5, phi=0.9273, theta=1.9513, mass=8.4262))

True

In [14]:
# Test instance type for any level of granularity.
(
    isinstance(vector.obj(x=1.1, y=2.2), vector.Vector),                                    # is a vector or array of vectors
    isinstance(vector.obj(x=1.1, y=2.2), vector.Vector2D),                                  # is 2D (not 3D or 4D)
    isinstance(vector.obj(x=1.1, y=2.2), vector.VectorObject),                              # is a vector object (not an array)
    isinstance(vector.obj(px=1.1, py=2.2), vector.Momentum),                                # has momentum synonyms
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Planar),                      # has transverse plane (2D, 3D, or 4D)
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Spatial),                     # has all spatial coordinates (3D or 4D)
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4), vector.Lorentz),                     # has temporal coordinates (4D)
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).azimuthal, vector.AzimuthalXY),       # azimuthal coordinate type
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).longitudinal, vector.LongitudinalZ),  # longitudinal coordinate type
    isinstance(vector.obj(x=1.1, y=2.2, z=3.3, t=4.4).temporal, vector.TemporalT),          # temporal coordinate type
)

(True, True, True, True, True, True, True, True, True, True)

The allowed keyword arguments for 2D vectors are:

   * `x` and `y` for Caresian azimuthal coordinates,
   * `px` and `py` for momentum,
   * `rho` and `phi` for polar azimuthal coordinates,
   * `pt` and `phi` for momentum.

For 3D vectors, you need the above and:

   * `z` for the Cartesian longitudinal coordinate,
   * `pz` for momentum,
   * `theta` for the spherical polar angle (from $0$ to $\pi$, inclusive),
   * `eta` for pseudorapidity, which is a kind of spherical polar angle.

For 4D vectors, you need the above and:

   * `t` for the Cartesian temporal coordinate,
   * `E` or `energy` to get four-momentum,
   * `tau` for the "proper time" (temporal coordinate in the vector's rest coordinate system),
   * `M` or `mass` to get four-moemtum.

Since momentum vectors have momentum-synonyms _in addition_ to the geometrical names, any momentum-synonym will make the whole vector a momentum vector.

If you want to bypass the dimension and coordinate system inference through keyword arguments (e.g. for static typing), you can use specialized constructors:

In [15]:
vector.VectorObject2D.from_xy(1.1, 2.2)

vector.obj(x=1.1, y=2.2)

In [16]:
vector.MomentumObject3D.from_rhophiz(1.1, 2.2, 3.3)

vector.obj(pt=1.1, phi=2.2, pz=3.3)

In [17]:
vector.VectorObject4D.from_xyetatau(1.1, 2.2, 3.3, 4.4)

vector.obj(x=1.1, y=2.2, eta=3.3, tau=4.4)

and so on, for all combinations of azimuthal, longitudinal, and temporal coordinates, geometric and momentum-flavored.

### NumPy arrays of vectors

In [18]:
# NumPy-like arguments (literally passed through to NumPy)
vector.array([
    (1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)
], dtype=[("x", float), ("y", float)])

VectorNumpy2D([(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [19]:
# Pandas-like arguments (dict from names to column arrays)
vector.array({"x": [1.1, 1.2, 1.3, 1.4, 1.5], "y": [2.1, 2.2, 2.3, 2.4, 2.5]})

VectorNumpy2D([(1.1, 2.1), (1.2, 2.2), (1.3, 2.3), (1.4, 2.4), (1.5, 2.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [20]:
# As with objects, the coordinate system and dimension is taken from the names of the fields.
vector.array({
    "x": [1.1, 1.2, 1.3, 1.4, 1.5],
    "y": [2.1, 2.2, 2.3, 2.4, 2.5],
    "z": [3.1, 3.2, 3.3, 3.4, 3.5],
    "t": [4.1, 4.2, 4.3, 4.4, 4.5],
})

VectorNumpy4D([(1.1, 2.1, 3.1, 4.1), (1.2, 2.2, 3.2, 4.2), (1.3, 2.3, 3.3, 4.3),
               (1.4, 2.4, 3.4, 4.4), (1.5, 2.5, 3.5, 4.5)],
              dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('t', '<f8')])

In [21]:
vector.array({
    "pt": [1.1, 1.2, 1.3, 1.4, 1.5],
    "phi": [2.1, 2.2, 2.3, 2.4, 2.5],
    "eta": [3.1, 3.2, 3.3, 3.4, 3.5],
    "M": [4.1, 4.2, 4.3, 4.4, 4.5],
})

MomentumNumpy4D([(1.1, 2.1, 3.1, 4.1), (1.2, 2.2, 3.2, 4.2), (1.3, 2.3, 3.3, 4.3),
                 (1.4, 2.4, 3.4, 4.4), (1.5, 2.5, 3.5, 4.5)],
                dtype=[('rho', '<f8'), ('phi', '<f8'), ('eta', '<f8'), ('tau', '<f8')])

Existing NumPy arrays can be viewed as arrays of vectors, but it needs to be a [structured array](https://numpy.org/doc/stable/user/basics.rec.html) with recognized field names.

In [22]:
# NumPy array         # interpret groups of four values as named fields              # give it vector properties and methods
np.arange(0, 24, 0.1).view([("x", float), ("y", float), ("z", float), ("t", float)]).view(vector.VectorNumpy4D)

VectorNumpy4D([( 0. ,  0.1,  0.2,  0.3), ( 0.4,  0.5,  0.6,  0.7),
               ( 0.8,  0.9,  1. ,  1.1), ( 1.2,  1.3,  1.4,  1.5),
               ( 1.6,  1.7,  1.8,  1.9), ( 2. ,  2.1,  2.2,  2.3),
               ( 2.4,  2.5,  2.6,  2.7), ( 2.8,  2.9,  3. ,  3.1),
               ( 3.2,  3.3,  3.4,  3.5), ( 3.6,  3.7,  3.8,  3.9),
               ( 4. ,  4.1,  4.2,  4.3), ( 4.4,  4.5,  4.6,  4.7),
               ( 4.8,  4.9,  5. ,  5.1), ( 5.2,  5.3,  5.4,  5.5),
               ( 5.6,  5.7,  5.8,  5.9), ( 6. ,  6.1,  6.2,  6.3),
               ( 6.4,  6.5,  6.6,  6.7), ( 6.8,  6.9,  7. ,  7.1),
               ( 7.2,  7.3,  7.4,  7.5), ( 7.6,  7.7,  7.8,  7.9),
               ( 8. ,  8.1,  8.2,  8.3), ( 8.4,  8.5,  8.6,  8.7),
               ( 8.8,  8.9,  9. ,  9.1), ( 9.2,  9.3,  9.4,  9.5),
               ( 9.6,  9.7,  9.8,  9.9), (10. , 10.1, 10.2, 10.3),
               (10.4, 10.5, 10.6, 10.7), (10.8, 10.9, 11. , 11.1),
               (11.2, 11.3, 11.4, 11.5), (11.6, 11.7, 11.8, 11

Since `VectorNumpy2D`, `VectorNumpy3D`, `VectorNumpy4D`, and their momentum equivalents are NumPy array subclasses, all of the normal NumPy methods and functions work on them.

In [23]:
np.arange(0, 24, 0.1).view([("x", float), ("y", float), ("z", float), ("t", float)]).view(vector.VectorNumpy4D).reshape(6, 5, 2)

VectorNumpy4D([[[( 0. ,  0.1,  0.2,  0.3), ( 0.4,  0.5,  0.6,  0.7)],
                [( 0.8,  0.9,  1. ,  1.1), ( 1.2,  1.3,  1.4,  1.5)],
                [( 1.6,  1.7,  1.8,  1.9), ( 2. ,  2.1,  2.2,  2.3)],
                [( 2.4,  2.5,  2.6,  2.7), ( 2.8,  2.9,  3. ,  3.1)],
                [( 3.2,  3.3,  3.4,  3.5), ( 3.6,  3.7,  3.8,  3.9)]],

               [[( 4. ,  4.1,  4.2,  4.3), ( 4.4,  4.5,  4.6,  4.7)],
                [( 4.8,  4.9,  5. ,  5.1), ( 5.2,  5.3,  5.4,  5.5)],
                [( 5.6,  5.7,  5.8,  5.9), ( 6. ,  6.1,  6.2,  6.3)],
                [( 6.4,  6.5,  6.6,  6.7), ( 6.8,  6.9,  7. ,  7.1)],
                [( 7.2,  7.3,  7.4,  7.5), ( 7.6,  7.7,  7.8,  7.9)]],

               [[( 8. ,  8.1,  8.2,  8.3), ( 8.4,  8.5,  8.6,  8.7)],
                [( 8.8,  8.9,  9. ,  9.1), ( 9.2,  9.3,  9.4,  9.5)],
                [( 9.6,  9.7,  9.8,  9.9), (10. , 10.1, 10.2, 10.3)],
                [(10.4, 10.5, 10.6, 10.7), (10.8, 10.9, 11. , 11.1)],
                

All of the keyword arguments and rules that apply to `vector.obj` construction apply to `vector.array` dtypes.

Geometrical names are used in the dtype, even if momentum-synonyms are used in construction.

In [24]:
vector.array({"px": [1, 2, 3, 4], "py": [1.1, 2.2, 3.3, 4.4], "pz": [0.1, 0.2, 0.3, 0.4]})

MomentumNumpy3D([(1., 1.1, 0.1), (2., 2.2, 0.2), (3., 3.3, 0.3), (4., 4.4, 0.4)],
                dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8')])

### Awkward Arrays of vectors

[Awkward Arrays](https://awkward-array.org/) are arrays with more complex data structures than NumPy allows, such as variable-length lists, nested records, missing and even heterogeneous data (multiple data types: use sparingly).

The `vector.Array` function behaves exactly like the [ak.Array](https://awkward-array.readthedocs.io/en/latest/_auto/ak.Array.html) constructor, except that it makes arrays of vectors.

In [25]:
vector.Array([
    [{"x": 1, "y": 1.1, "z": 0.1}, {"x": 2, "y": 2.2, "z": 0.2}],
    [],
    [{"x": 3, "y": 3.3, "z": 0.3}],
    [{"x": 4, "y": 4.4, "z": 0.4}, {"x": 5, "y": 5.5, "z": 0.5}, {"x": 6, "y": 6.6, "z": 0.6}],
])

<VectorArray3D [[{x: 1, y: 1.1, z: 0.1, ... z: 0.6}]] type='4 * var * Vector3D["...'>

If you want _any_ records named "`Vector2D`", "`Vector3D`", "`Vector4D`", "`Momentum2D`", "`Momentum3D`", or "`Momentum4D`" to be interpreted as vectors, register the behaviors globally.

In [26]:
vector.register_awkward()

In [27]:
ak.Array([
    [{"x": 1, "y": 1.1, "z": 0.1}, {"x": 2, "y": 2.2, "z": 0.2}],
    [],
    [{"x": 3, "y": 3.3, "z": 0.3}],
    [{"x": 4, "y": 4.4, "z": 0.4}, {"x": 5, "y": 5.5, "z": 0.5}, {"x": 6, "y": 6.6, "z": 0.6}],
],
    with_name="Vector3D"
)

<VectorArray3D [[{x: 1, y: 1.1, z: 0.1, ... z: 0.6}]] type='4 * var * Vector3D["...'>

All of the keyword arguments and rules that apply to `vector.obj` construction apply to `vector.Array` field names.

## Vector properties

Any geometrical coordinate can be computed from vectors in any coordinate system; they'll be provided or computed as needed.

In [28]:
vector.obj(x=3, y=4).rho

5.0

In [29]:
vector.obj(rho=5, phi=0.9273).x

2.9999808719721477

In [30]:
vector.obj(rho=5, phi=0.9273).y

4.000014345949428

In [31]:
vector.obj(x=1, y=2, z=3).theta

0.6405223126794245

In [32]:
vector.obj(x=1, y=2, z=3).eta

1.1035868415601453

Some properties are not coordinates, but derived from them.

In [33]:
vector.obj(x=1, y=2, z=3).costheta

0.8017837257372732

In [34]:
vector.obj(x=1, y=2, z=3).mag   # spatial magnitude

3.7416573867739413

In [35]:
vector.obj(x=1, y=2, z=3).mag2   # spatial magnitude squared

14

These properties are provided because they can be computed faster or with more numerical stability in different coordinate systems. For instance, the magnitude ignores `phi` in polar coordinates.

In [36]:
vector.obj(rho=3, phi=0.123456789, z=4).mag2

25

Momentum vectors have geometrical properties as well as their momentum-synonyms.

In [37]:
vector.obj(px=3, py=4).rho

5.0

In [38]:
vector.obj(px=3, py=4).pt

5.0

In [39]:
vector.obj(x=1, y=2, z=3, E=4).tau

1.4142135623730951

In [40]:
vector.obj(x=1, y=2, z=3, E=4).mass

1.4142135623730951

Here's the key thing: _arrays of vectors return arrays of coordinates_.

In [41]:
vector.array({
    "x": [1.0, 2.0, 3.0, 4.0, 5.0],
    "y": [1.1, 2.2, 3.3, 4.4, 5.5],
    "z": [0.1, 0.2, 0.3, 0.4, 0.5],
}).theta

array([1.50363023, 1.50363023, 1.50363023, 1.50363023, 1.50363023])

In [42]:
vector.Array([
    [{"x": 1, "y": 1.1, "z": 0.1}, {"x": 2, "y": 2.2, "z": 0.2}],
    [],
    [{"x": 3, "y": 3.3, "z": 0.3}],
    [{"x": 4, "y": 4.4, "z": 0.4}, {"x": 5, "y": 5.5, "z": 0.5}],
]).theta

<Array [[1.5, 1.5], [], [1.5], [1.5, 1.5]] type='4 * var * float64'>

In [43]:
# Make a large, random NumPy array of 3D momentum vectors.
array = np.random.normal(0, 1, 150).view([(x, float) for x in ("x", "y", "z")]).view(vector.MomentumNumpy3D).reshape(5, 5, 2)
array

MomentumNumpy3D([[[(-7.50610146e-01,  1.0425774 ,  0.29778433),
                   ( 1.13287769e+00, -0.24836654,  0.19102889)],
                  [( 3.23256593e-01, -0.82910565,  0.79786817),
                   (-9.39658863e-01, -0.29452071, -0.10450563)],
                  [(-3.78822438e-01, -0.68410682,  0.82435938),
                   (-4.55054092e-01,  0.75207561, -1.56669745)],
                  [(-9.99329869e-01, -0.25884826,  0.20012055),
                   ( 1.64173331e-01, -1.41145348,  1.06922038)],
                  [( 3.59727840e-01,  0.97302169,  2.74344962),
                   ( 4.87818109e-02,  0.81457248, -1.32574577)]],

                 [[(-7.60756991e-01,  0.51675723,  1.55833683),
                   ( 3.45371784e-01, -0.53088786, -0.89626629)],
                  [( 2.63637828e-01,  1.39933396,  0.81886247),
                   (-2.28332543e-01, -1.72884423, -0.67500397)],
                  [( 7.61498620e-01, -1.94529145, -0.94960896),
                   (-4.07648082

In [44]:
# Get the transverse momentum of each one.
array.pt

array([[[1.28467242, 1.15978351],
        [0.88989382, 0.98473409],
        [0.78199014, 0.87902898],
        [1.03230936, 1.42096932],
        [1.03738871, 0.81603186]],

       [[0.919668  , 0.63334318],
        [1.4239524 , 1.74385726],
        [2.08902824, 2.85988478],
        [1.9150139 , 1.74972858],
        [0.99711068, 1.16542579]],

       [[1.26848639, 0.54252751],
        [1.72071232, 0.17785796],
        [0.41264633, 1.40481731],
        [1.54562798, 0.90618722],
        [0.96968051, 2.54524905]],

       [[1.29613304, 1.23542338],
        [0.61551705, 1.29741116],
        [1.10402231, 1.80376837],
        [1.09538891, 1.35164312],
        [0.80560085, 1.46978187]],

       [[0.05977147, 1.6111476 ],
        [0.50954712, 0.04813401],
        [1.35301714, 0.4839862 ],
        [1.08929993, 0.75920338],
        [0.24656454, 0.27739146]]])

In [45]:
# The array and its components have the same shape.
array.shape

(5, 5, 2)

In [46]:
array.pt.shape

(5, 5, 2)

In [47]:
# Make a large, random Awkward Array of 3D momentum vectors.
array = vector.Array([[{x: np.random.normal(0, 1) for x in ("px", "py", "pz")} for inner in range(np.random.poisson(1.5))] for outer in range(50)])
array

<MomentumArray3D [[{x: 0.346, y: -0.338, ... z: -1.02}]] type='50 * var * Moment...'>

In [48]:
# Get the transverse momentum of each one, in the same nested structure.
array.pt

<Array [[0.484, 1.04], [1.27, ... 1.6], [1.23]] type='50 * var * float64'>

In [49]:
# The array and its components have the same list lengths (and can therefore be used together in subsequent calculations).
ak.num(array)

<Array [2, 1, 2, 3, 0, 2, ... 1, 0, 1, 1, 2, 1] type='50 * int64'>

In [50]:
ak.num(array.pt)

<Array [2, 1, 2, 3, 0, 2, ... 1, 0, 1, 1, 2, 1] type='50 * int64'>

## Vector methods

Vector methods require arguments (in parentheses), which may be scalars or other vectors, depending on the calculation.

In [51]:
vector.obj(x=3, y=4).rotateZ(0.1)

vector.obj(x=2.585678829246765, y=4.279516911052588)

In [52]:
vector.obj(rho=5, phi=0.4).rotateZ(0.1)

vector.obj(rho=5, phi=0.5)

In [53]:
# Broadcasts a scalar rotation angle of 0.5 to all elements of the NumPy array.
print(vector.array({"rho": [1, 2, 3, 4, 5], "phi": [0.1, 0.2, 0.3, 0.4, 0.5]}).rotateZ(0.5))

[(1., 0.6) (2., 0.7) (3., 0.8) (4., 0.9) (5., 1. )]


In [54]:
# Matches each rotation angle to an element of the NumPy array.
print(vector.array({"rho": [1, 2, 3, 4, 5], "phi": [0.1, 0.2, 0.3, 0.4, 0.5]}).rotateZ(np.array([0.1, 0.2, 0.3, 0.4, 0.5])))

[(1., 0.2) (2., 0.4) (3., 0.6) (4., 0.8) (5., 1. )]


In [55]:
# Broadcasts a scalar rotation angle of 0.5 to all elements of the Awkward Array.
print(vector.Array([[{"rho": 1, "phi": 0.1}, {"rho": 2, "phi": 0.2}], [], [{"rho": 3, "phi": 0.3}]]).rotateZ(0.5))

[[{rho: 1, phi: 0.6}, {rho: 2, phi: 0.7}], [], [{rho: 3, phi: 0.8}]]


In [56]:
# Broadcasts a rotation angle of 0.1 to both elements of the first list, 0.2 to the empty list, and 0.3 to the only element of the last list.
print(vector.Array([[{"rho": 1, "phi": 0.1}, {"rho": 2, "phi": 0.2}], [], [{"rho": 3, "phi": 0.3}]]).rotateZ([0.1, 0.2, 0.3]))

[[{rho: 1, phi: 0.2}, {rho: 2, phi: 0.3}], [], [{rho: 3, phi: 0.6}]]


In [57]:
# Matches each rotation angle to an element of the Awkward Array.
print(vector.Array([[{"rho": 1, "phi": 0.1}, {"rho": 2, "phi": 0.2}], [], [{"rho": 3, "phi": 0.3}]]).rotateZ([[0.1, 0.2], [], [0.3]]))

[[{rho: 1, phi: 0.2}, {rho: 2, phi: 0.4}], [], [{rho: 3, phi: 0.6}]]


Some methods are equivalent to binary operators.

In [58]:
vector.obj(x=3, y=4).scale(10)

vector.obj(x=30, y=40)

In [59]:
vector.obj(x=3, y=4) * 10

vector.obj(x=30, y=40)

In [60]:
10 * vector.obj(x=3, y=4)

vector.obj(x=30, y=40)

In [61]:
vector.obj(rho=5, phi=0.5) * 10

vector.obj(rho=50, phi=0.5)

Some methods involve more than one vector.

In [62]:
vector.obj(x=1, y=2).add(vector.obj(x=5, y=5))

vector.obj(x=6, y=7)

In [63]:
vector.obj(x=1, y=2) + vector.obj(x=5, y=5)

vector.obj(x=6, y=7)

In [64]:
vector.obj(x=1, y=2).dot(vector.obj(x=5, y=5))

15

In [65]:
vector.obj(x=1, y=2) @ vector.obj(x=5, y=5)

15

The vectors can use different coordinate systems. Conversions are necessary, but minimized for speed and numeric stability.

In [66]:
vector.obj(x=3, y=4) @ vector.obj(x=6, y=8)   # both are Cartesian, dot product is exact

50

In [67]:
vector.obj(rho=5, phi=0.9273) @ vector.obj(x=6, y=8)   # one is polar, dot product is approximate

49.99999999942831

In [68]:
vector.obj(x=3, y=4) @ vector.obj(rho=10, phi=0.9273)   # one is polar, dot product is approximate

49.99999999942831

In [69]:
vector.obj(rho=5, phi=0.9273) @ vector.obj(rho=10, phi=0.9273)   # both are polar, a formula that depends on phi differences is used

50.0

The vectors can be from different backends. Normal rules for broadcasting Python numbers, NumPy arrays, and Awkward Arrays apply.

In [70]:
vector.array({"x": [1, 2, 3, 4, 5], "y": [0.1, 0.2, 0.3, 0.4, 0.5]}) + vector.obj(x=10, y=5)

VectorNumpy2D([(11., 5.1), (12., 5.2), (13., 5.3), (14., 5.4), (15., 5.5)],
              dtype=[('x', '<f8'), ('y', '<f8')])

In [74]:
(
    vector.Array([                                                   # an Awkward Array of vectors
        [{"x": 1, "y": 1.1}, {"x": 2, "y": 2.2}],
        [],
        [{"x": 3, "y": 3.3}],
        [{"x": 4, "y": 4.4}, {"x": 5, "y": 5.5}],
    ])
    + vector.obj(x=10, y=5)                                          # and a single vector object
)

<VectorArray2D [[{x: 11, y: 6.1}, ... x: 15, y: 10.5}]] type='4 * var * Vector2D...'>

In [75]:
(
    vector.Array([                                                   # an Awkward Array of vectors
        [{"x": 1, "y": 1.1}, {"x": 2, "y": 2.2}],
        [],
        [{"x": 3, "y": 3.3}],
        [{"x": 4, "y": 4.4}, {"x": 5, "y": 5.5}],
    ])
    + vector.array({"x": [4, 3, 2, 1], "y": [0.1, 0.1, 0.1, 0.1]})   # and a NumPy array of vectors
)

<VectorArray2D [[{x: 5, y: 1.2}, ... x: 6, y: 5.6}]] type='4 * var * Vector2D["x...'>